# Preprocessing
This notebook is responsible for transformation of raw data collected from Spotify's API and adjusting the structure and data types for storage in a relational database.

In [1]:
# Import necessary packages

import os
import sqlite3
import pandas as pd
import ast
import json

In [2]:
# Load the dataset
df = pd.read_csv('../data/raw/spotify_playlists.csv')

## Data Extraction
I begin with extracting only the information I am interested in and creating separate data frames that later will be transformed into database tables.

In [3]:
df['playlist_id'] = df['playlist_href'].apply(lambda x: x.split('/')[-1])
df['songs'] = df['songs'].apply(lambda x: ast.literal_eval(x))
# chatGPT helped in the line below
df['songs'] = [songs + [{'playlist_id': playlist_id}] for songs, playlist_id in zip(df['songs'], df['playlist_id'])]

In [4]:
# Copy was made to avoid SettingWithCopyWarning
playlists = df[['playlist_id', 'name', 'description', 'image_url']].copy()
playlists['num_followers'] = df['followers']

Helper functions below are used to create a readily formatted data frame with songs details. `artist_list_to_string` creates a String of artists' names separated by commas (I will use SQLite that cannot store lists), `clean_song_data` extracts only the relevant information for every song.

In [5]:
def artist_list_to_string(artists):
    str = ""

    for artist in artists:
        str += artist['name'] + ", "

    str = str[:-2]
    return str
    
def clean_song_data(song: list, playlist_id):
    row = {}
    song = song['track']

    # Some tracks are unavailable in the country associated with the account
    if song == None:
        return row
    
    row['song_id'] = song['external_ids']['isrc']
    row['playlist_id'] = playlist_id
    row['album_id'] = song['album']['id']
    row['title'] = song['name']
    row['release_date'] = pd.to_datetime(song['album']['release_date'], format='ISO8601')
    row['is_explicit'] = song['explicit']
    row['album_name'] = song['album']['name']
    row['artists'] = artist_list_to_string(song['artists'])
    row['popularity'] = song['popularity']

    return row

After creating new functions we are ready to apply them to transform the songs data into a more managable format.

In [6]:
# chatGPT helped in the line below
songs = df['songs'].apply(lambda song_list: [clean_song_data(song, song_list[-1]['playlist_id']) for song in song_list[:-1]])
songs = songs.explode().apply(pd.Series)

# To protect the data frame from the unavailable tracks, I remove rows that have all values equal to NaN
songs = songs.dropna(how='all')

## Structuring the data
To remove redundant records and minimize the storage required, data frames are further transormed. That includes removal of duplicates and splitting into smaller tables. 

### Check for duplicates
First, we investigate the presence of duplicates. Uniqueness of every playlist is ensured by the return statement of the `get_playlists_links` function in the notebook NB01. Naturally, it should be expected that the same song could be present in multiple playlists. To accomodate future exploratory data analysis, we make a new variable representing the number of appearances a song has in all playlists considered.

In [7]:
# The output of this cell motivates the creation of value counts variable
songs['num_occurrences'] = songs['song_id'].map(songs['song_id'].value_counts())
songs.loc[songs['num_occurrences'] > 1].set_index('num_occurrences').head(3)

,song_id,playlist_id,album_id,title,release_date,is_explicit,album_name,artists,popularity
num_occurrences,,,,,,,,,
2,QZWBK2200043,37i9dQZF1DX8womvTyUjrN,2GHyrL2mwHWmw1vmQfCrNv,Ké MaL,2024-07-26,False,Ké MaL,Elsa y Elmar,2.0
3,USUM72406076,37i9dQZF1DX8womvTyUjrN,27ZCBzP3A8nAMjr2E7JjtB,¿Para Qué?,2024-07-17,False,¿Para Qué?,Ela Taubert,60.0
6,USUG12400968,37i9dQZF1DX8womvTyUjrN,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,False,Si Antes Te Hubiera Conocido,KAROL G,94.0


Additionally, multiple songs can come from the same album. To reduce the amount of storage space required, I split the `songs` table into two parts: variables related strictly to songs and related to the albums.

I expect to have duplicates in the `albums` and `songs` data frames. To resolve the future issue of dupicates, only one of the duplicates is kept before we proceed further. Also, due to [licensing reasons](https://developer.spotify.com/documentation/web-api/concepts/track-relinking) it is possible that one album may have two or more different IDs. To accomodate for that fact I also keep only one of the album observations that differ only by the ID number.

As the next step, I create a table that represents the relation between the songs and the albums they are part of. Similar relation holds for songs and playlists, which is reflected in the code below. 

In [8]:
# The playlist table was created at the top of the notebook
albums = songs[['album_id', 'album_name', 'release_date']].copy()

# Mappings between songs, albums and playlists
song_album_mapping = songs[['song_id', 'album_id']].copy()
song_album_mapping = song_album_mapping.drop_duplicates()

song_playlist_mapping = songs[['song_id', 'playlist_id']].copy()
song_playlist_mapping = song_playlist_mapping.drop_duplicates()

# Removal of duplicates
albums = albums.drop_duplicates(subset='album_id')
albums = albums.drop_duplicates(subset=['album_name', 'release_date'])

songs = songs.drop_duplicates(subset='song_id')
# After additional inspection, I found 5 songs that are double counted
# As the same song was member of two identical albums (licensing reasons)
songs = songs.loc[~songs['song_id'].isin({'USUM72317268', 'GBAHT1901121', 'USAT21001985', 'NLF712406166', 'GMM881200003'})]


# Removal of redundant columns
songs = songs.drop(columns=['album_id', 'album_name', 'release_date', 'playlist_id'])

After obtaining all data frames of interest, I check whether they have any missing values.

In [9]:
print(albums.isna().sum())
print(songs.isna().sum())
print(playlists.isna().sum())
print(song_album_mapping.isna().sum())
print(song_playlist_mapping.isna().sum())

album_id        0
album_name      0
release_date    0
dtype: int64
song_id            0
title              0
is_explicit        0
artists            0
popularity         0
num_occurrences    0
dtype: int64
playlist_id      0
name             0
description      0
image_url        0
num_followers    0
dtype: int64
song_id     0
album_id    0
dtype: int64
song_id        0
playlist_id    0
dtype: int64


By now, my data frames are well-structured, without duplicates and missing values. It is time to prepare a database for their storage.

# Creation of a Relational Database
Having cleaned and divided the data into meaningful tables, I move on to creating a database to store all the information. As mentioned previously, I will be using [SQLite](https://www.sqlite.org/index.html) because the database can be stored locally and the limitation of maximum one open connection is not an issue for this project.

In [10]:
%load_ext sql
%config SqlMagic.autocommit=True

### Modification of Data Types
To ensure efficient and reliable storage of information in the database, I change the column types to minimize the memory space assigned to store the values. My type choices are based on a manual inspection of the values every column takes and the otuput of the cell below. For example, `album_id` is always a 22-character string, whereas the lengths of `album_names` vary but the longest name has 84 characters. In case you are curious, it is the 'Spider-Man: Into the Spider-Verse (Soundtrack From & Inspired by the Motion Picture)' album.

In [11]:
def check_max_length(df, col):
    max = df[col].apply(lambda x: len(list(x))).max()
    print(f'Max length of a string in the {col} column is {max}.')
    
check_max_length(albums, 'album_name')
check_max_length(songs, 'title')
check_max_length(songs, 'artists')

Max length of a string in the album_name column is 118.
Max length of a string in the title column is 143.
Max length of a string in the artists column is 311.


Below, all tables' schemas are created with appropriate choice of data types and primary keys. Additionally, uniqueness constraints were added were possible to enable faster data retrival and foreign key constraints were added to ensure integrity of references.

In [12]:
%%sql sqlite:///../data/clean/spotify_playlists.db --alias db

CREATE TABLE albums
(
    album_id CHAR(22) PRIMARY KEY,
    album_name VARCHAR(84),
    release_date DATE,
    UNIQUE (album_name, release_date)
);

CREATE TABLE songs
(
    song_id CHAR(12) PRIMARY KEY,
    title VARCHAR(106),
    is_explicit BOOLEAN,
    artists VARCHAR(70),
    popularity TINYINT,
    num_occurrences TINYINT
);

CREATE TABLE song_album_map
(
    song_id CHAR(12),
    album_id CHAR(22),
    PRIMARY KEY (song_id, album_id),
    FOREIGN KEY (song_id) REFERENCES songs(song_id),
    FOREIGN KEY (album_id) REFERENCES albums(album_id)
    
);

CREATE TABLE song_playlist_map
(
    song_id CHAR(12),
    playlist_id CHAR(22),
    PRIMARY KEY (song_id, playlist_id),
    FOREIGN KEY (song_id) REFERENCES songs(song_id),
    FOREIGN KEY (playlist_id) REFERENCES playlists(playlist_id)
);

CREATE TABLE playlists
(
    playlist_id CHAR(22) PRIMARY KEY,
    name VARCHAR(50),
    description VARCHAR(100),
    num_followers INT,
    image_url CHAR(64),
    UNIQUE (name, description, image_url)
);

Connecting to 'db'

++
||
++
++

Now, I can populate the tables with the cleaned data.

In [13]:
conn = sqlite3.connect('../data/clean/spotify_playlists.db')

albums.to_sql('albums', conn, if_exists='append', index=False)
songs.to_sql('songs', conn, if_exists='append', index=False)
playlists.to_sql('playlists', conn, if_exists='append', index=False)
song_album_mapping.to_sql('song_album_map', conn, if_exists='append', index=False)
song_playlist_mapping.to_sql('song_playlist_map', conn, if_exists='append', index=False)

3890